In [1]:
## REMEMBER TO ACTIVATE `GPU_optimization (aGPUo)`

import tensorflow
import torch
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scienceplots

from matplotlib import animation
from matplotlib.animation import PillowWriter

plt.style.use(['science', 'notebook'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

2025-05-19 15:19:12.198169: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-19 15:19:12.198353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-19 15:19:12.201726: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-19 15:19:12.221616: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-19 15:19:14.051876: W tensorflow/compiler/tf2

## TWO STREAM INSTABILITY

Consider N fixed ions and N moving electrons as an one-dimensional unmagnetized plasma (B=0). The equations of motion for this system are

$$
    d_t r_i = v_i;\quad d_t v_i = -\frac{e}{m_e}E(r_i)
$$

For the numerical solution of this system, it is important to notice that it is not necessary to discretize positions nor velocities; the discretization
in space is only for the solution of the Poisson equation. **Therefore there is no truncation error regarding the positions and velocities of the electrons; it is only rounding error.**

Posson equation states that

$$
    d^2_x \Phi(x) = -\frac{e}{m_e}(n_o-n(x)); \quad d_x\Phi(x) = - E(x),
$$

with $n_o = N/L = <n(x)>$ being the uniform ion number density; and, $n(x)$ is the electron number density. Now, consider two counter-propagating Maxwellian beams of mean speed $v_b$ and thermal velocity $v_{th}$; thus, 

$$
    f(x,v) = \frac{n_o}{2\sqrt{2\pi}v){th}}\left[exp\left(-\frac{1}{2v^2_{th}}\left(v - v_b\right)^2\right) + exp\left(-\frac{1}{2v^2_{th}}\left(v + v_b\right)^2\right)\right].
$$

With this, one can obtain $f(x,v)dxdv$ the number of electrons in between $x$ + $x+dx$ with velocities $v$ and $v + dv$. Also, one can obtain the electron number density by integrating over the entire velocity space. The definition of the thermal velocity is

$$
    v_{th} = \sqrt{\frac{k_bT}{m_e}}
$$

Where $k_b$ is the Boltzmann constant. It is important to know that if $v_b >> v_{th}$ , then, the Maxwellian distribution becomes unstable to the well-known two-stream instability . It is possible to investigate this instability numerically.

### PERIODIC BOUNDARY CONDITIONS

By considering periodic boundary contions, one can easily see that for $0\leq x \leq L$:

* $n(0) = n(L)$
* $E(0) = E(L)$
* $\Phi(0) = \Phi(L)$
* Electrons that crosses the right boundary should reappear in the left boundary and vice versa.

To reduce rounding error, one can consider the normalized variables: normalize time with respect the plasma frequency $\omega_p$ and the Debye length $\lambda_D$
for spatial quantities:

$$
    \omega^2_p = \frac{n_oe^2}{\epsilon_o m_e}; \quad \lambda_D = \frac{v_{th}}{\omega_p}
$$

Thus, the set of equations that describe the system become:

* $d_t r_i = v_i$
* $d_t v_i = -E(r_i)$
* $d_x\Phi(x) = -E(x)$
* $d^2_x\Phi(x) = n(x)/n_o - 1$
* $v_{th}$ = 1

And the distribution function takes the form of 

$$
    f(x,v) = \frac{n_o}{2\sqrt{2\pi}}\left[ exp\left(-\frac{1}{2}\left(v-v_b\right)^2\right) + exp\left(-\frac{1}{2}\left(v+v_b\right)^2\right) \right]
$$

# MODEL

### EQUATIONS OF MOTION 

For the equations of motions, we will be considering the 4-th order Runge Kutta (RK4).

Runge-Kutta methods solve initial value problems (IVP). In the context of multivariate calculus, IVP are is ordinary differential equations with initial conditions which specify the value of the unknown functions at a given point in the domain ([recommended to check the existence and uniqueness of solutions](https://www.math.iitb.ac.in/~siva/ma41707/ode3.pdf)). The different types of Runge-Kutta methods, can be observed in the following image. 

![alt text](Images/Runge-Kutta.png "Gebraeuchliche Varienten des Runge-Kutta-Verfahrens")

### PARTICLE DENSITY $n(x)$

To be able to solve the equations of motions, it is necesary to know the electric field $E(x)$ and the electric potential $\Phi(x)$. This is where the Particle in Cell Method comes into play.  